In [0]:
!pip install PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

fileId = drive.CreateFile({'id': '#'}) 
print(fileId['title'])  # dataset.zip
fileId.GetContentFile('temp.zip')  # Save Drive file as a local file

!unzip temp.zip -d ./

imdb_master.zip
Archive:  temp.zip
replace ./imdb_master.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: ./imdb_master.csv       


In [0]:
# GloVe file

fileId = drive.CreateFile({'id': '#'}) 
print(fileId['title'])  # dataset.zip
fileId.GetContentFile('glove6b100dtxt.zip')  # Save Drive file as a local file

!unzip glove6b100dtxt.zip -d ./


glove6b100dtxt.zip
Archive:  glove6b100dtxt.zip
  inflating: ./glove.6B.100d.txt     


In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
from sklearn.model_selection import train_test_split
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [0]:
df = pd.read_csv('imdb_master.csv', encoding = "ISO-8859-1")
del df['Unnamed: 0']
del df['file']
del df['type']

In [0]:
df = df.loc[0:49999]
df = df.sample(frac=1).reset_index(drop=True)

In [0]:
def clean_reviews(text):
    lemmatizer = WordNetLemmatizer()
    my_stopwords = stopwords.words('english') 
    text = text.replace("<br >", "")
    text = text.replace("</br >", "")        
    text = re.sub('[^a-zA-Z]',' ', text)
    text = text.lower() 
    text = [lemmatizer.lemmatize(token) for token in text.split(" ")]
    text = [lemmatizer.lemmatize(token, "v") for token in text]
    text = [word for word in text if not word in my_stopwords]
    text = " ".join(text)
    return text  

In [0]:
df["review"]=df.review.apply(lambda x: clean_reviews(x))

In [0]:
df.head()

,review,label
0,remember old kung fu movie use watch friday sa...,neg
1,stick around one brief series film pair bobb...,neg
2,discovery channel animal planet must ashamed ...,neg
3,like many western pennsylvania history buff r...,neg
4,best original show see year watch fall love ...,pos


In [0]:
df["label"] = df["label"].map({'pos': 1, 'neg': 0})
df.head()

,review,label
0,remember old kung fu movie use watch friday sa...,0
1,stick around one brief series film pair bobb...,0
2,discovery channel animal planet must ashamed ...,0
3,like many western pennsylvania history buff r...,0
4,best original show see year watch fall love ...,1


In [0]:
embeddings_index = dict()
f = open('glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [0]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation, Bidirectional
from keras.layers.embeddings import Embedding
import string

In [0]:
vocabulary_size = 10000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(df['review'])
sequences = tokenizer.texts_to_sequences(df['review'])
data = pad_sequences(sequences, maxlen=150)


embedding_matrix = np.zeros((vocabulary_size, 100))
for word, index in tokenizer.word_index.items():
    if index > vocabulary_size - 1:
        break
    else:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector

In [0]:
model_glove = Sequential()
model_glove.add(Embedding(vocabulary_size, 100, input_length=150, weights=[embedding_matrix], trainable=False))
model_glove.add(LSTM(200, dropout=0.2, recurrent_dropout=0.2))
model_glove.add(Dense(1, activation='sigmoid'))
model_glove.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model_glove.fit(data, df['label'], validation_split=0.2, epochs = 10)

Instructions for updating:
Use tf.cast instead.
Train on 40000 samples, validate on 10000 samples
Epoch 1/10
40000/40000 [==============================] - 677s 17ms/step - loss: 0.5049 - acc: 0.7508 - val_loss: 0.3781 - val_acc: 0.8360
Epoch 2/10
40000/40000 [==============================] - 668s 17ms/step - loss: 0.3843 - acc: 0.8299 - val_loss: 0.3580 - val_acc: 0.8445
Epoch 3/10
40000/40000 [==============================] - 668s 17ms/step - loss: 0.3467 - acc: 0.8490 - val_loss: 0.3179 - val_acc: 0.8660
Epoch 4/10
40000/40000 [==============================] - 662s 17ms/step - loss: 0.3189 - acc: 0.8613 - val_loss: 0.3010 - val_acc: 0.8703
Epoch 5/10
40000/40000 [==============================] - 663s 17ms/step - loss: 0.2979 - acc: 0.8718 - val_loss: 0.3259 - val_acc: 0.8612
Epoch 6/10
40000/40000 [==============================] - 584s 15ms/step - loss: 0.2796 - acc: 0.8810 - val_loss: 0.2996 - val_acc: 0.8656
Epoch 7/10
40000/40000 [==============================] - 320s 8ms/s

In [0]:
model_glove = Sequential()
model_glove.add(Embedding(vocabulary_size, 100, input_length=150, weights=[embedding_matrix], trainable=False))
model_glove.add(Dropout(0.2))
model_glove.add(Conv1D(64, 5, activation='relu'))
model_glove.add(MaxPooling1D(pool_size=4))
model_glove.add(Bidirectional(LSTM(200, dropout=0.2, recurrent_dropout=0.2)))
model_glove.add(Dense(1, activation='sigmoid'))
model_glove.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model_glove.fit(data, df['label'], validation_split=0.2, epochs = 10)

Train on 40000 samples, validate on 10000 samples
Epoch 1/10
40000/40000 [==============================] - 296s 7ms/step - loss: 0.4807 - acc: 0.7685 - val_loss: 0.4014 - val_acc: 0.8178
Epoch 2/10
40000/40000 [==============================] - 294s 7ms/step - loss: 0.3873 - acc: 0.8270 - val_loss: 0.3565 - val_acc: 0.8407
Epoch 3/10
40000/40000 [==============================] - 293s 7ms/step - loss: 0.3572 - acc: 0.8414 - val_loss: 0.3379 - val_acc: 0.8523
Epoch 4/10
40000/40000 [==============================] - 294s 7ms/step - loss: 0.3359 - acc: 0.8549 - val_loss: 0.3401 - val_acc: 0.8544
Epoch 5/10
40000/40000 [==============================] - 293s 7ms/step - loss: 0.3203 - acc: 0.8609 - val_loss: 0.3256 - val_acc: 0.8593
Epoch 6/10
40000/40000 [==============================] - 292s 7ms/step - loss: 0.3091 - acc: 0.8674 - val_loss: 0.3194 - val_acc: 0.8648
Epoch 7/10
40000/40000 [==============================] - 293s 7ms/step - loss: 0.2955 - acc: 0.8732 - val_loss: 0.3247 - 